# Predicting Rainfall Anomalies in the Amazon
## Model Outline:
- Preprocessing
- Multivariate EOF
- Ridge Regression
- Lasso Regression
- Neural Network Regression

Created by David Gotler, 2021

## Preprocessing
### Define a dataframe class to:
- Load in the datasets:
    - feature dataset (ERA5): variables are CAPE, CIN, geopotential, relative humidity
        - lat: (0,-30), lon: (-40,-80)
    - target dataset (GPCC): variable is precipitation
        - lat: (-5,-25), lon: (-50,-70)
- Detrend the dataset by removing linear trends. We want the dataset to be stationary
- Calculate standardized anomalies by subtracting the monthly mean and dividing by standard deviation. We want centered dataset with unit variance.

In [1]:
import numpy as np
import xarray as xr

from matplotlib import pyplot as plt
    
from dataframe import dataframe
# directories
data_dir = 'data/'
features_fname = 'ERA5.features.1950-2019.nc'
target_fname = 'precip.mon.total.1x1.v2020.nc'

# input regions
lat_ft = -30,0
lon_ft = -80,-40
lat_t = -20,-5
lon_t = -70,-50
yrs = 1950,2019
# pack tuple
range_ft =  lat_ft+lon_ft+yrs
range_t = lat_t+lon_t+yrs

# Create df object to store our feature and target datasets
df = dataframe(data_dir,features_fname,target_fname,features_range=range_ft,target_range=range_t)
df.detrend()
df.std_anom()
df.flatten()
df.remove_nan()

/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:169: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:169: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:169: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:169: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:169: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:169: RankWarnin

In [4]:
from forecast import get_dataset
from forecast import DirectForecaster,RecursiveForecaster
from sklearn.linear_model import LinearRegression
forecaster = DirectForecaster(LinearRegression(),lags=3,target_months=[9,10,11],steps=3,include_autoreg=True,avg_lags=False,pca_features=False)
X_train,y_train,X_test,y_test = forecaster.train_test_split(4,df.target_da,df.features_da)
forecaster.fit(X_train,y_train)

y_pred = forecaster.predict(X_test,y_test)
# plots = forecaster.data_plots(y_test,y_pred)
# plots['facet']
get_dataset(forecaster.get_training_matrix(9,0,X_train,y_train)[0])

DEBUG:root:Coordinates of da:
[]
[<xarray.DataArray 'variable' (lag_feature: 234672)>
array(['precip', 'precip', 'precip', ..., 'cin', 'cin', 'cin'],
      dtype=object)
Coordinates:
    variable     (lag_feature) object 'precip' 'precip' 'precip' ... 'cin' 'cin'
    lat          (lag_feature) float64 -19.5 -19.5 -19.5 -19.5 ... 0.0 0.0 0.0
    lon          (lag_feature) float64 -69.5 -68.5 -67.5 ... -40.5 -40.25 -40.0
  * lag_feature  (lag_feature) MultiIndex
  - month        (lag_feature) int64 6 6 6 6 6 6 6 6 6 6 ... 8 8 8 8 8 8 8 8 8 8
  - feature      (lag_feature) int64 0 1 2 3 4 ... 78219 78220 78221 78222 78223, <xarray.DataArray 'lat' (lag_feature: 234672)>
array([-19.5, -19.5, -19.5, ...,   0. ,   0. ,   0. ])
Coordinates:
    variable     (lag_feature) object 'precip' 'precip' 'precip' ... 'cin' 'cin'
    lat          (lag_feature) float64 -19.5 -19.5 -19.5 -19.5 ... 0.0 0.0 0.0
    lon          (lag_feature) float64 -69.5 -68.5 -67.5 ... -40.5 -40.25 -40.0
  * lag_feature  